In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
cd sample_data

/content/sample_data


In [3]:
'''
get the churn data set from https://github.com/samarthmaiya/ML/raw/master/
'''
!wget https://github.com/samarthmaiya/ML/raw/master/uts-mc/churn.zip

--2021-07-17 08:54:01--  https://github.com/samarthmaiya/ML/raw/master/uts-mc/churn.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/samarthmaiya/ML/master/uts-mc/churn.zip [following]
--2021-07-17 08:54:02--  https://raw.githubusercontent.com/samarthmaiya/ML/master/uts-mc/churn.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156460 (153K) [application/zip]
Saving to: ‘churn.zip’

churn.zip           100%[===================>] 152.79K  --.-KB/s    in 0.01s   

2021-07-17 08:54:02 (10.4 MB/s) - ‘churn.zip’ saved [156460/156460]



In [4]:
import zipfile
with zipfile.ZipFile('churn.zip', 'r') as zip_ref:
  zip_ref.extractall('sample_data')

In [5]:
df = pd.read_csv('sample_data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df.dropna(inplace = True)

In [8]:
class onehotencoder:    
    
    def __init__(self, dataframe,encodingvalue):
        self.encode(dataframe,encodingvalue)
        
    def coloumnencoder(self ,i , encodingvalue):
      '''
      param coloumn from data fram
      param encoded set value
      return string to numeric encoded data
      '''
    #encodingvalue =  [('Yes' , 1), ('No' , 0)]
      #outlayer data 
      retundata = 9
      for k,v in encodingvalue:   
          if(i == k):
              retundata =  v
      return retundata;
    
    def encode(self,dataframe, encodingvalue):  
        '''
        helper function to prepare dataframe
        param dataframe
        param encoding value
        return  null
        '''
        index = 0
        for i in dataframe:    
            #print(i,self.coloumnencoder(i),index)
            dataframe[index] = self.coloumnencoder(i ,encodingvalue )
            index +=1


In [9]:
def preparedictionary(datafram ,coloumnname):
    '''
    param input dataframe
    param input coloumn name
    return dictionary of unquie numeric value with resepect to string
    '''
    dicdata = []
    index = 0
    for i in datafram[coloumnname].unique():
        dicdata.append((i,index))
        index +=1
    #print("coloumnname ::" , coloumnname , dicdata)
    return dicdata

In [10]:
coltobeencoded = ['gender', 'Partner', 'Dependents','PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod','Churn' ]

for i in coltobeencoded:
    dictdata = preparedictionary(df,i)
    onehotencoder(df[i][:],dictdata)

In [11]:
df.to_csv('pre-processesed churn.csv',index=False)

In [12]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,29.85,29.85,0
1,5575-GNVDE,1,0,1,0,34,1,1,0,1,1,1,0,0,0,1,1,1,56.95,1889.5,0
2,3668-QPYBK,1,0,1,0,2,1,1,0,1,0,0,0,0,0,0,0,1,53.85,108.15,1
3,7795-CFOCW,1,0,1,0,45,0,0,0,1,1,1,1,0,0,1,1,2,42.30,1840.75,0
4,9237-HQITU,0,0,1,0,2,1,1,1,0,1,0,0,0,0,0,0,0,70.70,151.65,1


In [13]:
#drop null value if any from data set
df.isnull().values.any()
df.dropna(inplace =True)

In [14]:
#split the data for train and validation set based on user input , 
#future scpoe introduce random selector for generalization
#test_size is in the range of 0.1 to 1
def train_test_split(dataframe,test_size=0.1):
    totalsize = len(dataframe)
    testdatasize = int(test_size * totalsize)
    
    Xtraindata = dataframe[testdatasize:]
    Xtestdata =  dataframe[:testdatasize]
    
    return Xtraindata, Xtestdata

In [15]:
 Xtraindata, Xtestdata = train_test_split(df)

In [16]:
print('train data shape ::::',Xtraindata.shape)
print('test data shape ::::',Xtestdata.shape)

train data shape :::: (6339, 21)
test data shape :::: (704, 21)


In [17]:
class Node:
    '''
    single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

In [18]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=3):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(data):
        '''
        :param s: list
        :return: float, entropy value
        '''
        coll = list(data)
        _dict = {x:coll.count(x)/len(coll) for x in coll}
        _prob = np.array(list(_dict.values()))
        return - _prob.dot(np.log2(_prob))
    
    def _info_gain(self, parent, l_child, r_child):
        '''
        Information Gain = entropy(parent) – [average entropy(left_child) + average entropy(right_child)]
        
        :param parent: list, parent node
        :param l_child: list, left child of a parent
        :param r_child: list, right child of a parent
        :return: float, information gain
        '''
        p_len = len(parent)
        left_avg = len(l_child) /p_len 
        right_avg = len(r_child) /p_len
        
        return self._entropy(parent) - (left_avg * self._entropy(l_child) + right_avg * self._entropy(r_child))
    
    def _best_split(self, X, y):
        '''        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # for each feature
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            # for very unique feature value
            for currentth in np.unique(X_curr):
                # split data set to the left and right parts
                # Left includes records lower or equal to the threshold
                # Right includes records higher than the threshold
                p= np.concatenate((X, y.reshape(1, -1).T), axis=1)
                leftdata = []
                rightdata = []
                for row in p:
                  if (row[f_idx] <= currentth):
                    leftdata.append(row)
                for row in p :
                  if (row[f_idx] > currentth):
                    rightdata.append(row)
                p_left = np.array(leftdata)
                p_right = np.array(rightdata)

                # calculate only if there's data in both subsets
                if len(p_left) > 0 and len(p_right) > 0:
                    # prepare subsets
                    y = p[:, -1]
                    y_left = p_left[:, -1]
                    y_right = p_right[:, -1]

                    # Caclulate the information gain 
                    # save the split parameters only if the current split if better then the previous best
                    gain = self._info_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'threshold': currentth,
                            'feature_index': f_idx,  
                            'df_right': p_right,                          
                            'df_left': p_left,                            
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        '''
         recursively iterate to build decession tree Node     
        :param X: np.array, features
        :param y: np.array or list, target
        :param depth: current depth of a tree
        :return: Node
        '''
        n_rows, n_cols = X.shape
        
        # Check to see if a node should be leaf node
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            # Get the best split
            bestsplit = self._best_split(X, y)
            # If the split isn't pure
            if bestsplit['gain'] > 0:
                # Build a tree on the left
                left = self._build(
                    X=bestsplit['df_left'][:, :-1], 
                    y=bestsplit['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=bestsplit['df_right'][:, :-1], 
                    y=bestsplit['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=bestsplit['feature_index'], 
                    threshold=bestsplit['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=bestsplit['gain']
                )
        # Leaf node - value 
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        '''
        train DT model.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: None
        '''
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):
        '''
        predict a data.
        
        :param x: single observation
        :param tree: built tree
        :return: float, predicted class
        '''
        # Leaf node
        if tree.value != None:
            return tree.value
        f_score = x[tree.feature]
        
        # left
        if f_score <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # right
        if f_score > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        classify new data.
        
        :param X: np.array, features
        :return: np.array, predicted classes
        '''
        return [self._predict(x, self.root) for x in X]


In [19]:
s = DecisionTree()

In [20]:
Xtrain = Xtraindata.iloc[:, 1:20]
Ytrain = Xtraindata.iloc[:,20]

In [21]:
xtest = Xtestdata.iloc[:, 1:20]
ytest = Xtestdata.iloc[:,20]

In [22]:
a = Xtrain[:6000].to_numpy()
b = Ytrain[:6000].to_numpy()

In [ ]:
s.fit(a,b)

In [ ]:
a1 = xtest.iloc[:700].to_numpy()
b1 = ytest.iloc[:700].to_numpy()

In [ ]:
preddata = np.array(s.predict(a1))

In [ ]:
def accuracy(actual,predicted):
    '''
    function to calculate accuracy
    param : actual array data
    param : predicted array data
    return accuaracy value
    '''
    yhat=actual
    y=predicted
    acc=np.mean(y==yhat)
    return acc

In [ ]:
accuracy(b1,preddata)